<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myProspects_Upside_Analysis_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance==0.2.59
!pip install ta

In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os
project_path = '/content/drive/My Drive/quantvesting'
os.chdir(project_path)

import warnings
warnings.filterwarnings('ignore')

from google.colab import data_table

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Import the function from the script
import myProspects_Upside_Analysis_v2

all = 1

# Call the function to get the DataFrame
df_prospects_features = myProspects_Upside_Analysis_v2.get_prospects_features_v2(all)
cols = ['Symbol', 'Close', 'FTT',  'Dev%_200', 'Dev%_PE', 'Conviction', 'RSI_14', 'RSP', 'Today%', 'FTT%', 'ATH%','Gained%', 'Criteria', 'Strategy', 'Sales_Grwth%', 'Profit_Grwth%', 'Category', 'InFolio', 'MCap', 'MBQ', 'ROE%', 'CFO_2_EBITDA%', 'ROE%/PE']
df_prospects_features = df_prospects_features[cols].drop_duplicates()

Run date time (IST): 2025-06-06 15:10:22
-------------------


ERROR:yfinance:$ISEC.NS: possibly delisted; no price data found  (period=1mo)


get_prospects_features
qualified stocks: 267
with latest results: 76
still star stocks: 54


In [4]:
cols = ['Symbol', 'FTT', 'Dev%_200', 'Dev%_PE', 'Conviction', 'RSI_14', 'RSP', 'FTT%', 'ATH%','Gained%', 'CumlRnk','ROE%/PE', 'Criteria', 'Strategy', 'Category', 'InFolio']

tmp_df = df_prospects_features
list_x40 = list(tmp_df[tmp_df['MBQ'].str.contains('X40', na=False)]['Symbol'].values)
list_x5k = list(tmp_df[tmp_df['MBQ'].str.contains('X5K', na=False)]['Symbol'].values)
list_x40n = list(tmp_df[tmp_df['MBQ'].str.contains('X40N', na=False)]['Symbol'].values)
list_hc = list(tmp_df[tmp_df['Conviction'].isin(['X-LC', 'H-LC', 'X-MC', 'X-SC'])]['Symbol'].values)
list_ox40 = list(tmp_df[tmp_df['MBQ'].str.contains('OX40', na=False)]['Symbol'].values)
list_ox40n = list(tmp_df[tmp_df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)
#len(set(list_x40 + list_x5k) - set( list_x40n))
stock_list = list(set(list(set(list_x40 + list_x5k + list_hc)- set(list_ox40 + list_ox40n))))

In [5]:
# get the cumulative rank
df_tmp = df_prospects_features # [df_prospects_features['Symbol'].isin(stock_list)]

df_tmp['Rnk_Dev%_200'] = df_tmp['Dev%_200'].rank(method='min', ascending=True)
df_tmp['Rnk_Dev%_PE'] = df_tmp['Dev%_PE'].rank(method='min', ascending=True)
df_tmp['Rnk_Gained%'] = df_tmp['Gained%'].rank(method='min', ascending=True)
df_tmp['Rnk_FTT%'] = df_tmp['FTT%'].rank(method='max', ascending=False)
df_tmp['Rnk_Sales_Grwth%'] = df_tmp['Sales_Grwth%'].rank(method='max', ascending=False)
df_tmp['Rnk_Profit_Grwth%'] = df_tmp['Profit_Grwth%'].rank(method='max', ascending=False)
df_tmp['Rnk_MCap'] = df_tmp['MCap'].rank(method='max', ascending=False)
df_tmp['Rnk_ROE%'] = df_tmp['ROE%'].rank(method='max', ascending=False)
df_tmp['Rnk_RSI_14'] = df_tmp['RSI_14'].rank(method='max', ascending=False)
df_tmp['Rnk_CFO_2_EBITDA%'] = df_tmp['CFO_2_EBITDA%'].rank(method='max', ascending=False)
df_tmp['Rnk_ROE%/PE'] = df_tmp['ROE%/PE'].rank(method='max', ascending=False)
df_tmp['Ovrl_Rank'] = df_tmp['Rnk_Dev%_200'] + df_tmp['Rnk_Dev%_PE'] + df_tmp['Rnk_Gained%'] + df_tmp['Rnk_FTT%'] + df_tmp['Rnk_Sales_Grwth%'] + df_tmp['Rnk_Profit_Grwth%'] + df_tmp['Rnk_MCap'] + df_tmp['Rnk_CFO_2_EBITDA%'] + df_tmp['Rnk_ROE%'] + df_tmp['Rnk_RSI_14'] + df_tmp['Rnk_ROE%/PE']
df_tmp['Cat_Rank'] = df_tmp.groupby('Conviction')['Ovrl_Rank'].rank(ascending=True, method='min')
priority_map = {'X-LC': 0,'H-LC': 1,'X-MC': 2,'X-SC': 3,'M-LC': 4,'H-MC': 5,'H-SC': 6,'L-LC': 7,'M-MC': 8,'M-SC': 9,'L-MC': 10,'L-SC': 11}
df_tmp['Conviction_Priority'] = df_tmp['Conviction'].map(priority_map)
df_tmp = df_tmp.sort_values(by=['Conviction_Priority', 'Cat_Rank']).reset_index(drop=True)
df_tmp['CumlRnk'] = df_tmp.index + 1
# Drop the helper columns
df_tmp.drop(columns=['Rnk_Dev%_200','Rnk_Dev%_PE','Rnk_Gained%','Rnk_FTT%','Rnk_Sales_Grwth%','Rnk_Profit_Grwth%','Rnk_MCap','Rnk_ROE%','Rnk_RSI_14','Rnk_CFO_2_EBITDA%'], inplace=True)
df_prospects_features = df_tmp[cols]
data_table.DataTable(df_prospects_features.sort_values(by = 'CumlRnk', ascending=True), include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
0,TCS,4998.00,-13.18,-19.02,X-LC,42.0,17.29,47.83,31.09,5.55,1,2.1,X40,BTT,IT,1.0
1,INFY,2275.00,-10.82,-12.78,X-LC,56.0,39.47,45.68,26.28,13.38,2,1.2,X40,BTT,IT,1.0
2,LTIM,7197.33,-5.63,-2.76,X-LC,73.0,77.07,38.40,38.40,29.34,3,0.6,X200,ATH,IT,1.0
3,BAJAJ-AUTO,12666.40,-6.55,51.54,X-LC,56.0,53.76,46.77,46.77,18.11,4,0.7,X40,ATH,AUTO,NaN
4,ASIANPAINT,4250.00,-12.44,-23.22,X-LC,36.0,3.38,89.44,55.91,5.22,5,0.4,X40,BTT,PAINTS,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,SPORTKING,157.26,12.13,NaN,NaN,61.0,62.03,33.57,33.57,64.26,264,NaN,AR,ATH,TEXTILES,NaN
264,TASTYBITE,20387.66,-2.96,NaN,NaN,56.0,74.44,97.40,97.40,32.66,265,NaN,XR,ATH,FMCG,NaN
265,V2RETAIL,2031.65,18.97,NaN,NaN,50.0,53.38,11.32,11.32,242.79,266,NaN,AR,ATH,TEXTILES,NaN
266,VEEDOL,3065.39,-6.28,NaN,NaN,57.0,75.94,85.11,85.11,22.07,267,NaN,XR,ATH,CHEMICALS,NaN


In [6]:
# module to print date timestamp
def read_csv(csv_file_name):
  csv_file_path = os.path.join(project_path, 'data', csv_file_name)
  df = pd.read_csv(csv_file_path)
  return df

mypps_df = read_csv('myProspectsScrips.csv')

# Suppose this is your updated DataFrame with new cuml_rnk values
cols1 = ['Symbol', 'CumlRnk']
df_tmp = df_prospects_features[cols1]

# Merge the two DataFrames on 'symbol', keeping the CSV structure
mypps_df = mypps_df.merge(df_tmp[['Symbol', 'CumlRnk']], on='Symbol', how='left', suffixes=('', '_new'))

# Replace 'cuml_rnk' with the new value only if it exists in df
mypps_df['CumlRnk'] = mypps_df['CumlRnk_new'].combine_first(mypps_df['CumlRnk'])

# Drop the helper column
mypps_df.drop(columns=['CumlRnk_new'], inplace=True)

# Save back to CSV
#mypps_df.to_csv(project_path+'/data/myProspectsScrips.csv', index=False)

In [7]:
# High quality (X40 + X40N + X5K + X-LC + H-LC + X-MC + X-SC) prospects
df_tmp = df_prospects_features[df_prospects_features['Symbol'].isin(stock_list)]
df_tmp = df_tmp[(df_tmp['FTT%']>20) & (df_tmp['Dev%_PE']<0)]
data_table.DataTable(df_tmp.sort_values(by = 'CumlRnk', ascending=True), include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
0,TCS,4998.00,-13.18,-19.02,X-LC,42.0,17.29,47.83,31.09,5.55,1,2.1,X40,BTT,IT,1.0
1,INFY,2275.00,-10.82,-12.78,X-LC,56.0,39.47,45.68,26.28,13.38,2,1.2,X40,BTT,IT,1.0
2,LTIM,7197.33,-5.63,-2.76,X-LC,73.0,77.07,38.40,38.40,29.34,3,0.6,X200,ATH,IT,1.0
4,ASIANPAINT,4250.00,-12.44,-23.22,X-LC,36.0,3.38,89.44,55.91,5.22,5,0.4,X40,BTT,PAINTS,1.0
13,SIEMENS,7969.85,-42.45,-2.32,X-LC,52.0,66.73,141.59,141.59,21.47,14,0.4,AR,ATH,ELECTRICAL,1.0
14,DMART,5201.00,2.74,-18.02,X-LC,59.0,40.23,23.77,28.31,23.44,15,0.1,X40N,NTT,FMCG,1.0
20,BAJAJFINSV,2392.00,9.36,-2.35,X-LC,48.0,13.53,20.38,61.13,30.25,21,0.4,X40,BTT,FINANCE,NaN
25,VBL,672.28,-15.75,-15.43,H-LC,42.0,3.76,40.98,40.98,9.56,26,0.4,X40N,ATH,FMCG,1.0
26,ABB,7934.00,-7.54,-33.83,H-LC,67.0,71.43,30.97,47.79,23.50,27,0.4,AR,NTT,ELECTRICAL,1.0
28,ITC,505.32,-4.97,-34.37,H-LC,50.0,16.17,20.03,20.03,8.59,29,1.8,X40,ATH,FMCG,NaN


In [8]:
# x24/x25
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XY24', 'X25'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
181,QUESS,986.0,-49.11,-30.58,M-SC,32.0,1.88,213.02,280.08,5.63,182,0.9,XY24,NTT,MISC,1.0
180,EASEMYTRIP,26.4,-26.73,11.35,M-SC,39.0,0.75,140.22,211.19,0.27,181,0.7,XY24,NTT,TRAVEL,1.0
118,BANDHANBNK,400.0,2.99,-32.41,H-SC,60.0,57.14,131.19,312.10,32.78,119,1.2,XY24,NTT,BANKS,1.0
124,REPCOHOME,880.0,-1.63,-54.55,H-SC,65.0,66.73,108.53,99.82,35.58,125,2.4,XY24,NTT,FINANCE,1.0
156,ACC,3906.0,-9.54,-35.47,M-MC,49.0,32.33,105.03,45.52,5.31,157,0.9,XY24,BTT,CEMENT,1.0
137,SFL,1287.0,-21.31,26.62,H-SC,49.0,19.55,98.72,204.52,3.89,138,0.0,XY24,NTT,MISC,1.0
198,GREENPANEL,537.0,-16.37,197.16,M-SC,66.0,78.57,97.57,120.64,24.76,199,0.2,XY24,NTT,MISC,1.0
87,AWL,485.0,-11.66,-68.69,H-MC,48.0,28.95,83.61,218.38,12.05,88,0.5,XY24,NTT,FMCG,1.0
213,SHALBY,327.0,-19.30,829.82,M-SC,37.0,28.20,79.30,76.38,1.16,214,0.1,XY24,NTT,HEALTHCARE,1.0
229,RBLBANK,388.0,19.36,41.00,M-SC,61.0,58.27,78.61,214.49,45.34,230,0.3,XY24,NTT,BANKS,1.0


In [9]:
# X-MC and X-SC prospects
df_tmp = df_prospects_features[df_prospects_features['Conviction'].str.startswith('X-', na=False)]
df_tmp = df_tmp[~df_tmp['Conviction'].isin(['X-LC'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
57,ALKYLAMINE,4567.52,3.84,-8.37,X-SC,69.0,81.20,130.30,130.30,26.44,58,0.3,SR,ATH,CHEMICALS,1.0
58,VAIBHAVGBL,521.00,-6.04,65.29,X-SC,58.0,84.21,105.36,267.27,26.47,59,0.4,XR,NTT,JEWELLERY,1.0
60,METROPOLIS,3102.00,-13.43,-3.54,X-SC,47.0,15.79,87.32,107.23,11.25,61,0.2,XR,NTT,HEALTHCARE,NaN
61,DEEPAKNTR,3102.60,-19.33,18.71,X-SC,42.0,19.92,59.86,59.86,7.38,62,0.4,AR,ATH,CHEMICALS,NaN
51,ASTRAL,2412.74,-5.08,-16.31,X-MC,65.0,81.95,58.17,58.17,23.21,52,0.2,AR,ATH,PLASTICS,NaN
47,COLPAL,3767.14,-14.71,0.61,X-MC,40.0,4.51,55.03,55.03,4.50,48,1.8,XY25,ATH,FMCG,1.0
50,DABUR,735.00,-9.36,-13.30,X-MC,49.0,21.43,52.71,37.98,6.62,51,0.3,XY24,BTT,FMCG,1.0
52,HONAUT,58518.52,-5.10,-18.69,X-MC,64.0,67.29,49.64,49.64,19.77,53,0.2,X40N,ATH,ELECTRICAL,1.0
48,HAVELLS,2069.16,-8.33,-8.02,X-MC,45.0,10.15,35.81,35.81,8.70,49,0.3,X40,ATH,ELECTRICAL,1.0
49,PGHH,18106.42,-9.46,-19.75,X-MC,35.0,6.02,34.12,34.12,5.55,50,1.3,X40,ATH,FMCG,NaN


In [10]:
# H-MC and H-SC prospects
df_tmp = df_prospects_features[df_prospects_features['Conviction'].str.startswith('H-', na=False)]
df_tmp = df_tmp[~df_tmp['Conviction'].isin(['H-LC'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
116,TANLA,1963.11,-4.63,-33.31,H-SC,70.0,95.49,214.75,214.75,49.30,117,1.5,AR,ATH,IT,1.0
120,RELAXO,1176.00,-24.90,-36.38,H-SC,61.0,63.16,161.86,216.95,12.30,121,0.1,X40N,NTT,FOOTWEAR,1.0
118,BANDHANBNK,400.00,2.99,-32.41,H-SC,60.0,57.14,131.19,312.10,32.78,119,1.2,XY24,NTT,BANKS,1.0
109,INDUSINDBK,1800.00,-21.94,69.29,H-MC,51.0,17.67,124.10,141.96,26.08,110,0.2,XR,NTT,BANKS,1.0
108,INDUSINDBK,1800.00,-19.84,72.86,H-MC,56.0,17.67,118.87,136.31,29.10,109,0.2,XR,NTT,BANKS,1.0
133,GLAND,3442.00,-0.77,-1.45,H-SC,73.0,80.08,109.21,159.67,18.12,134,0.2,XR,NTT,PHARMA,NaN
124,REPCOHOME,880.00,-1.63,-54.55,H-SC,65.0,66.73,108.53,99.82,35.58,125,2.4,XY24,NTT,FINANCE,1.0
113,SONATSOFTW,852.37,-21.42,-0.84,H-SC,59.0,45.49,107.79,107.79,36.23,114,1.0,MGC,ATH,IT,NaN
111,INDIAMART,4850.92,1.78,-50.71,H-SC,67.0,59.02,100.68,100.68,27.09,112,1.0,AR,ATH,MISC,1.0
137,SFL,1287.00,-21.31,26.62,H-SC,49.0,19.55,98.72,204.52,3.89,138,0.0,XY24,NTT,MISC,1.0


In [11]:
# x40
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
4,ASIANPAINT,4250.00,-12.44,-23.22,X-LC,36.0,3.38,89.44,55.91,5.22,5,0.4,X40,BTT,PAINTS,1.0
203,BATAINDIA,2096.00,-8.42,-32.45,M-SC,44.0,22.93,72.09,78.63,3.14,204,0.3,X40,NTT,FOOTWEAR,1.0
0,TCS,4998.00,-13.18,-19.02,X-LC,42.0,17.29,47.83,31.09,5.55,1,2.1,X40,BTT,IT,1.0
3,BAJAJ-AUTO,12666.40,-6.55,51.54,X-LC,56.0,53.76,46.77,46.77,18.11,4,0.7,X40,ATH,AUTO,NaN
1,INFY,2275.00,-10.82,-12.78,X-LC,56.0,39.47,45.68,26.28,13.38,2,1.2,X40,BTT,IT,1.0
48,HAVELLS,2069.16,-8.33,-8.02,X-MC,45.0,10.15,35.81,35.81,8.70,49,0.3,X40,ATH,ELECTRICAL,1.0
49,PGHH,18106.42,-9.46,-19.75,X-MC,35.0,6.02,34.12,34.12,5.55,50,1.3,X40,ATH,FMCG,NaN
107,ICICIPRULI,791.05,-3.20,-11.00,H-MC,54.0,50.00,24.38,24.38,18.52,108,0.1,X40,ATH,INSURANCE,1.0
130,SANOFI,7552.00,1.22,68.40,H-SC,52.0,41.73,22.68,60.71,25.68,131,1.3,X40,NTT,PHARMA,NaN
30,HCLTECH,2006.00,-4.32,12.43,H-LC,54.0,36.84,22.54,20.08,20.58,31,1.0,X40,BTT,IT,NaN


In [12]:
# x40n
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40N'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
120,RELAXO,1176.00,-24.90,-36.38,H-SC,61.0,63.16,161.86,216.95,12.30,121,0.1,X40N,NTT,FOOTWEAR,1.0
193,TEAMLEASE,3487.00,-23.63,155.01,M-SC,47.0,27.82,81.33,174.61,13.45,194,0.4,X40N,NTT,MISC,NaN
140,KAJARIACER,1541.71,-7.89,10.93,H-SC,61.0,92.86,52.16,52.16,29.99,141,0.2,X40N,ATH,CERAMICS,NaN
52,HONAUT,58518.52,-5.10,-18.69,X-MC,64.0,67.29,49.64,49.64,19.77,53,0.2,X40N,ATH,ELECTRICAL,1.0
112,BLUESTARCO,2337.55,-16.29,-15.29,H-SC,43.0,4.89,46.90,46.90,4.77,113,0.4,X40N,ATH,AC,NaN
72,BAJAJHFL,181.50,-1.13,-2.07,M-LC,57.0,44.74,44.55,44.55,18.74,73,0.3,X40N,ATH,NaN,NaN
25,VBL,672.28,-15.75,-15.43,H-LC,42.0,3.76,40.98,40.98,9.56,26,0.4,X40N,ATH,FMCG,1.0
90,DIXON,18940.15,-1.38,-35.52,H-MC,44.0,2.63,27.58,27.58,52.13,91,0.4,X40N,ATH,IT,NaN
115,LALPATHLAB,3545.00,-2.89,-32.03,H-SC,58.0,26.69,24.60,44.18,23.40,116,0.5,X40N,NTT,HEALTHCARE,1.0
14,DMART,5201.00,2.74,-18.02,X-LC,59.0,40.23,23.77,28.31,23.44,15,0.1,X40N,NTT,FMCG,1.0


In [13]:
# xr, sr, ar
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XR','SR', 'AR'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
250,BAJAJHIND,442.93,-16.66,NaN,NaN,63.0,93.98,1732.56,1732.56,38.04,251,NaN,XR,ATH,SUGAR,NaN
234,CENTRALBK,190.06,-20.42,-36.85,L-MC,61.0,72.56,377.66,377.66,17.51,235,1.2,XR,ATH,BANKS,NaN
247,ALOKINDS,79.81,-7.90,NaN,L-SC,59.0,83.83,312.67,312.67,29.97,248,NaN,AR,ATH,TEXTILES,NaN
248,APEX,849.16,-5.40,NaN,NaN,54.0,60.90,278.75,278.75,13.95,249,NaN,AR,ATH,MISC,NaN
190,IGL,707.00,-0.22,-7.97,M-SC,58.0,31.58,235.80,35.63,35.22,191,1.0,AR,BTT,GAS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,INDIGO,5589.00,17.26,24.95,M-LC,57.0,32.71,1.82,1.82,42.62,75,4.4,AR,ATH,AIRLINES,NaN
237,HINDPETRO,410.00,6.52,178.62,L-MC,55.0,30.45,0.76,10.22,38.52,238,1.1,XR,BTT,REFINERIES,NaN
55,SRF,3122.80,19.69,99.95,X-MC,66.0,46.62,0.00,0.00,45.86,56,0.1,XR,ATH,CHEMICALS,NaN
165,MUTHOOTFIN,2444.00,17.95,30.77,M-MC,71.0,54.14,0.00,0.00,44.05,166,1.1,XR,ATH,FINANCE,NaN
